#### 1° String de conexão (ajuste conforme seu ambiente)

In [7]:
from sqlalchemy import create_engine, text

server = 'DESKTOP-F0DI39E'
database = 'master'  # criaremos o banco a partir do master
driver = 'ODBC Driver 17 for SQL Server'

connection_string = (
    f"mssql+pyodbc://@{server}/{database}"
    f"?driver={driver.replace(' ', '+')}"
)

engine = create_engine(connection_string)


#### 2° Criar a base de dados via Python

In [ ]:
with engine.connect() as conn:
    conn.execute(text("""
        IF NOT EXISTS (SELECT name FROM sys.databases WHERE name = 'BI_Projeto')
        CREATE DATABASE BI_Projeto;
    """))
    conn.commit()


#### 3° Conectar na base criada

In [9]:
database = 'BI_Projeto'

connection_string = (
    f"mssql+pyodbc://@{server}/{database}"
    f"?driver={driver.replace(' ', '+')}"
)

engine = create_engine(connection_string)


#### Criação das tabelas dbo.Cliente

In [ ]:
create_table_sql = """
IF NOT EXISTS (
    SELECT * FROM sys.tables 
    WHERE name = 'dbo.Cliente'
)
BEGIN
    CREATE TABLE dbo.Cliente (
        id_Cliente INT IDENTITY(1,1) PRIMARY KEY,
        str_Cliente VARCHAR(150) NOT NULL
    );
END
"""

with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()


#### Criação das tabelas dbo.Usuário

In [ ]:
create_table_sql = """
IF NOT EXISTS (
    SELECT * FROM sys.tables 
    WHERE name = 'dbo.Usuario'
)
BEGIN
    CREATE TABLE dbo.Usuario (
    id_Usuario INT IDENTITY(1,1) PRIMARY KEY,
    id_Cliente INT NOT NULL,
    str_Usuario VARCHAR(150) NOT NULL,
    str_Email VARCHAR(150) NOT NULL,
    bit_StatusAtivo BIT NOT NULL,
    str_DescrInativo VARCHAR(255),

    CONSTRAINT FK_Usuario_Cliente
        FOREIGN KEY (id_Cliente)
        REFERENCES dbo.Cliente(id_Cliente)
);
END
"""

with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()

#### Inserir dados via Python dbo.Cliente

In [42]:
import pandas as pd
import pyodbc

conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-F0DI39E;"
    "DATABASE=BI_Projeto;"
    "Trusted_Connection=yes;"
)

cursor = conn.cursor()

# Truncate
cursor.execute("TRUNCATE TABLE dbo.Cliente")
conn.commit()

# Load Excel
df = pd.read_excel("base_projetos.xlsx", sheet_name="dbo.Cliente")

df.columns = df.columns.str.strip().str.lower()
print(df.columns)

for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO dbo.Cliente (str_Cliente)
        VALUES (?)
    """, row['str_cliente'])

conn.commit()
cursor.close()
conn.close()


Index(['id_cliente', 'str_cliente'], dtype='object')


#### Inserir dados via Python dbo.Usuario

In [ ]:
import pandas as pd
import pyodbc

conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=SEU_SERVIDOR;"
    "DATABASE=SEU_BANCO;"
    "Trusted_Connection=yes;"
)

cursor = conn.cursor()

# Truncate
cursor.execute("TRUNCATE TABLE dim_Cliente")
conn.commit()

# Load Excel
df = pd.read_excel("base_projetos.xlsx", sheet_name="Cliente")

for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO dim_Cliente (str_Cliente)
        VALUES (?)
    """, row['str_Cliente'])

conn.commit()
cursor.close()
conn.close()



#### (Opcional) Consultar os dados

In [45]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM dbo.Cliente"))
    for row in result:
        print(row)


(1, '000 - ATENDIMENTOS OW')
(2, '001 - singular medicamentos')


### OPICIONAL Atualizar com SQLAlchemy (caso você esteja usando)

In [33]:
from sqlalchemy import create_engine, text

engine = create_engine(
    "mssql+pyodbc://@localhost/BI_Projeto?driver=ODBC+Driver+17+for+SQL+Server"
)

with engine.connect() as conn:
    result = conn.execute(
        text("""
            UPDATE dbo.Cliente
            SET str_Cliente = :nome
            WHERE id_Cliente = :id
        """),
        {"nome": "001 - singular medicamentos", "id": 2}
    )
    conn.commit()


In [ ]:
import pandas as pd
import pyodbc

conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-F0DI39E;"
    "DATABASE=BI_Projeto;"
    "Trusted_Connection=yes;"
)

df = pd.read_excel("usuarios.xlsx")

cursor = conn.cursor()

for _, row in df.iterrows():
    # Busca id_Cliente
    cursor.execute("""
        SELECT id_Cliente
        FROM Cliente
        WHERE str_Cliente = ?
    """, row['str_Cliente'])

    result = cursor.fetchone()

    if result:
        id_cliente = result[0]

        cursor.execute("""
            INSERT INTO Usuario (
                id_Cliente,
                str_Usuario,
                str_Email,
                bit_StatusAtivo
            )
            VALUES (?, ?, ?, ?)
        """, id_cliente, row['str_Usuario'], row['str_Email'], row['bit_StatusAtivo'])

conn.commit()
conn.close()
